In [8]:
import sys
print(sys.executable)
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install accelerate
!{sys.executable} -m pip install transformers[torch]

c:\Users\Aarav\AppData\Local\Programs\Python\Python313\python.exe



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import json
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer
)
import random
import os

In [ ]:
# ─── HYPERPARAMETER TUNING ──────────────────────────────────────────────────────
def get_training_args(
    output_dir="./results",
    lr=2e-5,
    epochs=3,
    train_bs=16,
    eval_bs=32,
    weight_decay=0.0,
    warmup_steps=0
):
    from transformers import TrainingArguments
    return TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=train_bs,
        per_device_eval_batch_size=eval_bs,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        logging_dir="./logs",
        logging_steps=50,
    )
# ────────────────────────────────────────────────────────────────────────────────

In [ ]:
#load dataset
def load_dataset(path):
    samples = []
    with open(path, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                samples.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON on line {line_number}: {e}")
    return samples

In [ ]:
samples = []
decoder = json.JSONDecoder()

with open('training_code_snippets.jsonl', 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        idx = 0
        while idx < len(line):
            obj, consumed = decoder.raw_decode(line[idx:])
            samples.append(obj)
            idx += consumed
            while idx < len(line) and line[idx].isspace():
                idx += 1
vuln_types = sorted({s['vulnerability_type'] for s in samples})
label_map     = {v: i for i, v in enumerate(vuln_types)}
inv_label_map = {i: v for v, i in label_map.items()}

print("Label map:", label_map)
print("Inverse map:", inv_label_map)

Label map: {'Buffer Overflow': 0, 'Business Logic Errors': 1, 'CWE-1021: Clickjacking': 2, 'CWE-120: Buffer Overflow': 3, 'CWE-190: Integer Overflow': 4, 'CWE-190: Integer Overflow/Wraparound': 5, 'CWE-209: Exposure of Sensitive Info in Error Msgs': 6, 'CWE-209: Information Exposure': 7, 'CWE-20: Improper Input Validation': 8, 'CWE-22: Path Traversal': 9, 'CWE-276: Incorrect Default Permissions': 10, 'CWE-284: Improper Access Control': 11, 'CWE-287: Improper Authentication': 12, 'CWE-295: Improper Certificate Validation': 13, 'CWE-306: Unprotected API Endpoints': 14, 'CWE-326: Inadequate Encryption Strength': 15, 'CWE-326: Insecure Protocol Usage': 16, 'CWE-326: Weak Encryption': 17, 'CWE-352: Cross-Site Request Forgery (CSRF)': 18, 'CWE-362: Race Condition': 19, 'CWE-377: Unsafe Use of Temporary Files': 20, 'CWE-400: Resource Exhaustion': 21, 'CWE-400: Uncontrolled Resource Consumption': 22, 'CWE-416: Use After Free': 23, 'CWE-434: Unrestricted File Upload': 24, 'CWE-502: Insecure Des

In [ ]:
class CodeVulnerabilityDataset(Dataset):
    def __init__(self, samples, tokenizer, max_length=512):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        encoding = self.tokenizer(
            sample['code_snippet'],
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        label = label_map[sample['vulnerability_type']]
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

In [14]:
model_name = "microsoft/codebert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_map)
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ─── METRICS CALLBACK ───────────────────────────────────────────────────────────
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":      accuracy_score(labels, preds),
        "f1_weighted":   f1_score(labels, preds, average="weighted"),
    }
# ────────────────────────────────────────────────────────────────────────────────

In [ ]:
random.shuffle(samples)
split_idx = int(0.8 * len(samples))
train_samples = samples[:split_idx]
val_samples = samples[split_idx:]

train_dataset = CodeVulnerabilityDataset(train_samples, tokenizer)
val_dataset = CodeVulnerabilityDataset(val_samples, tokenizer)

In [17]:
import transformers
print(transformers.__version__)

from transformers import TrainingArguments
print(TrainingArguments.__init__.__code__.co_varnames)

4.51.3
('self', 'output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size', 'gradient_accumulation_steps', 'eval_accumulation_steps', 'eval_delay', 'torch_empty_cache_steps', 'learning_rate', 'weight_decay', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'max_grad_norm', 'num_train_epochs', 'max_steps', 'lr_scheduler_type', 'lr_scheduler_kwargs', 'warmup_ratio', 'warmup_steps', 'log_level', 'log_level_replica', 'log_on_each_node', 'logging_dir', 'logging_strategy', 'logging_first_step', 'logging_steps', 'logging_nan_inf_filter', 'save_strategy', 'save_steps', 'save_total_limit', 'save_safetensors', 'save_on_each_node', 'save_only_model', 'restore_callback_states_from_checkpoint', 'no_cuda', 'use_cpu', 'use_mps_device', 'seed', 'data_seed', 'jit_mode_eval', 'use_ipex', 'bf16', 'fp16', 'fp16_opt_level', 'half_precision_ba

In [ ]:
def get_training_args(
    output_dir: str = "./results",
    lr: float = 2e-5,
    epochs: int = 3,
    train_bs: int = 8,
    eval_bs: int = 16,
    weight_decay: float = 0.0,
    warmup_steps: int = 0,
    report_to: list = ["tensorboard"],
):
    return TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=train_bs,
        per_device_eval_batch_size=eval_bs,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        logging_dir="./logs",
        logging_strategy="steps",
        logging_steps=50,
        report_to=report_to,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
    )

C:\Users\Aarav\AppData\Local\Temp\ipykernel_4936\3359831958.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
training_args = get_training_args(
    lr=5e-4,
    epochs=10,
    train_bs=16,
    eval_bs=32,
    weight_decay=0.01,
    warmup_steps=500,
    report_to=["tensorboard"],
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
def predict(code: str):
    tokens = tokenizer(code, return_tensors="pt", truncation=True, padding=True)
    # Move inputs to model device
    device = next(model.parameters()).device
    tokens = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        outputs = model(**tokens)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()
    return inv_label_map[prediction]

In [ ]:
results = trainer.evaluate()
print(f"Eval accuracy: {results['eval_accuracy']:.4f}, F1: {results['eval_f1_weighted']:.4f}")

In [36]:
print(predict("os.system('cat ' + input())"))
print(predict("import os\nip = input('Enter IP: ')\nos.system('ping ' + ip)"))
print(predict("username = 'admin'\npassword = '1234'\nconnect(username, password)"))
print(predict("user_id = input('Enter user ID: ')\nquery = 'SELECT * FROM users WHERE id = ' + user_id\ncursor.execute(query)"))
print(predict("age = input('Enter your age: ')\nif age > 18:\n    print('Adult')"))
print(predict("import xml.etree.ElementTree as ET\ndata = input('XML: ')\nET.fromstring(data)"))
print(predict("del obj\nprint(obj.attribute)"))
print(predict("@app.route('/transfer', methods=['POST'])\ndef transfer():\n    do_transfer()"))


🚨 Vulnerability: CWE-77: Command Injection
🚨 Vulnerability: Command Injection
🚨 Vulnerability: CWE-798: Hard-coded Credentials
🚨 Vulnerability: SQL Injection
✅ Not vulnerable
🚨 Vulnerability: Path Traversal
🚨 Vulnerability: CWE-416: Use After Free
🚨 Vulnerability: CWE-79: Cross-site Scripting (XSS)
